In [ ]:
import json
import pandas as pd
import requests
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
print('Bibliotecas importadas')

In [ ]:
#municipios = "'Paraupebas'|'Maraba'|'Curionópolis'|'Canãa dos Carajas'|'Água Azul do Norte'|'Eldorado dos Carajas'
municipios = "1505536|1504208|1502772|1502152|1500347|1502954"

In [ ]:
url = f"https://servicodados.ibge.gov.br/api/v3/agregados/9606/periodos/-30/variaveis?localidades={municipios}&classificacao=287|2|86"

In [ ]:
print(url)

In [ ]:
payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

In [ ]:
data = json.loads(response.text)

In [ ]:
# Provided JSON data
# Deserializing into tables
tables = []
for item in data:
    for result in item['resultados']:
        classification = {classif['nome']: list(classif['categoria'].values())[0] for classif in result['classificacoes']}
        for serie in result['series']:
            series_data = serie['serie']
            localidade = serie['localidade']
            table_data = {**classification, **series_data, **localidade}
            tables.append(table_data)

# Converting tables to DataFrame
df = pd.DataFrame(tables)
print(df)


In [ ]:
df_base = df.melt(id_vars=['Idade', 'Sexo', 'Cor ou raça','id','nome'], var_name='ano', value_name='valor')

In [ ]:
df_base

In [ ]:
df_base.rename(columns={'Idade': 'idade', 'Sexo': 'sexo', 'Cor ou raça': 'cor_ou_raca', 'ano': 'ano', 'valor': 'valor', 'id':'id_ibge','nome':'municipio'}, inplace=True)

In [ ]:
#df_base.drop_duplicates(inplace=True)

In [ ]:
df_base

In [ ]:
#removendo valores nulos
df_base = df_base[df_base['valor'] != '-']
#removendo idades totais
df_base = df_base[df_base['idade'] != 'Total']
#removendo sexo total
df_base = df_base[df_base['sexo'] != 'Total']
#removendo cor ou raça total
df_base = df_base[df_base['cor_ou_raca'] != 'Total']
#remover nivel
df_base = df_base[df_base['ano'] != 'nivel']


In [ ]:
df_base

In [ ]:
# #removendo valores nulos
# df_t = df_base[df_base['valor'] != '-']
# #removendo idades totais
# df_t = df_t[df_t['idade'] == 'Total']
# #removendo sexo total
# df_t = df_t[df_t['sexo'] == 'Total']
# #removendo cor ou raça total
# df_t = df_t[df_t['cor_ou_raca'] == 'Total']


In [ ]:
#df_t

In [ ]:
df_base.drop_duplicates(inplace=True)

In [ ]:
df_base

In [ ]:
df_base['tipo_intervalo'] = df_base['idade'].apply(lambda x: True if ' a ' in x or ' ou mais' in x else False)


In [ ]:
df_base

In [ ]:
SQLALCHEMY = os.environ.get('SQLALCHEMY_PRODUCAO')

try:
    engine = create_engine(SQLALCHEMY)
    conn_hom = engine.connect()
    print('Conexão com banco de dados criada...')
except Exception as e:
    print(f'Falha ao criar conexão com banco de dados. {e}')

In [ ]:
df_base.to_csv('ibge_dados_mun.csv', index=False)

In [ ]:
df_base.to_sql('ibge_população_residente_segregado', schema='perfil_populacional', con=conn_hom, if_exists='append', index=False, method='multi' )